In [1]:
%load_ext autoreload
%autoreload 2
import litebird_sim as lbs
import numpy as np
import healpy as hp
import sbm
import logging
import toml
from pathlib import Path
logging.basicConfig(level=logging.WARNING);

In [2]:
CONFIG_PATH = Path.home() / ".config" / "litebird_imo"
CONFIG_FILE_PATH = CONFIG_PATH / "imo.toml"
tomlinfo = toml.load(CONFIG_FILE_PATH)
flatfile_location = tomlinfo["repositories"][0]["location"]

imo = lbs.Imo(flatfile_location=flatfile_location)

In [3]:
channel = sbm.channel_list[0]

config = sbm.Configlation(imo, channel)
config.mdim = 7
config.parallel = True
config.nside = 128

syst = sbm.Systematics()
syst.sigma_gain_T = 0.1
syst.sigma_gain_B = 0.1

syst.sigma_rho_T = np.deg2rad(1/60)
syst.sigma_rho_B = 0.

syst.sigma_chi_T = 0.
syst.sigma_chi_B = 0.

Here, we will take a `N` realisations for CMB, systematics variations and noise.


In [4]:
N = 2 # number of realizations
npix = hp.nside2npix(config.nside)
gain_maps = np.zeros([N,N,3,npix])
pointing_maps = np.zeros([N,N,3,npix])
noise_maps = np.zeros([N,3,npix])
fg_models = ["pysm_dust_1", "pysm_synch_1"]

for i in range(N):
    Mbsparams = lbs.MbsParameters(
        make_cmb=True,
        cmb_r=0.001,
        make_fg=True,
        seed_cmb=i,
        fg_models=fg_models,
        gaussian_smooth=False,
        bandpass_int=False,
        nside=config.nside,
        units="uK_CMB",
        maps_in_ecliptic=False,
    )
    syst.noise_seed = i
    for j in range(N):
        syst.syst_seed = j
        print(f"CMB seed: {i} | Noise seed {i} | Syst. seed: {j}")
        gain_map, noise_map, input_maps = sbm.sim_diff_gain_per_ch(config, syst, Mbsparams)
        pointing_map, noise_map, input_maps = sbm.sim_diff_pointing_per_ch(config, syst, Mbsparams)
        gain_maps[i,j] = gain_map
        pointing_maps[i,j] = pointing_map
        if i == 0:
            noise_maps[j] = noise_map

CMB seed: 0 | Noise seed 0 | Syst. seed: 0


Processing L1-040: 100%|██████████| 24/24 [00:03<00:00,  6.79it/s]


CMB seed: 0 | Noise seed 0 | Syst. seed: 1


Processing L1-040: 100%|██████████| 24/24 [00:03<00:00,  7.55it/s]


CMB seed: 1 | Noise seed 1 | Syst. seed: 0


Processing L1-040: 100%|██████████| 24/24 [00:03<00:00,  7.54it/s]


CMB seed: 1 | Noise seed 1 | Syst. seed: 1


Processing L1-040: 100%|██████████| 24/24 [00:02<00:00,  8.14it/s]
